# Contabilidade Vista & Revista

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all('h2', {'class': 'title'})
    a_links = []
    for item in results:
        try:
            remover_links = 'editorial' in item.find('a').text.lower()
            remover_links = remover_links or 'folha de rosto' in item.find('a').text.lower()
            remover_links = remover_links or 'dados da edição ' in item.find('a').text.lower()
            if not remover_links:
                a_links.append(item.find('a')['href'])
        except:
            pass
    return a_links

In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links

In [5]:
url_search_text = 'https://revistas.face.ufmg.br/index.php/contabilidadevistaerevista/search/search?query=__query__&searchJournal=1&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=score&orderDir=desc&searchPage=__pg_num__#results'

links = listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)

link = links[0]

link


'https://revistas.face.ufmg.br/index.php/contabilidadevistaerevista/article/view/1780'

In [6]:
links

['https://revistas.face.ufmg.br/index.php/contabilidadevistaerevista/article/view/1780',
 'https://revistas.face.ufmg.br/index.php/contabilidadevistaerevista/article/view/5162',
 'https://revistas.face.ufmg.br/index.php/contabilidadevistaerevista/article/view/3401',
 'https://revistas.face.ufmg.br/index.php/contabilidadevistaerevista/article/view/1828',
 'https://revistas.face.ufmg.br/index.php/contabilidadevistaerevista/article/view/3173',
 'https://revistas.face.ufmg.br/index.php/contabilidadevistaerevista/article/view/5276',
 'https://revistas.face.ufmg.br/index.php/contabilidadevistaerevista/article/view/1727']

In [7]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [9]:
def limpar_afiliacao(afiliacao):
    afiliacao = ' '.join(afiliacao.split())
    afiliacao = afiliacao.split(',')[0]
    afiliacao = afiliacao.split(' - ')[0]
    afiliacao = afiliacao.split('(')[0]
    afiliacao = afiliacao.split('/')[0]
    return afiliacao.strip()

In [10]:
def captura_titulo(soup):
    try:
        title = soup.find('h1', {'class': 'page_title'}).text
        title = ' '.join(title.split())
    except:
        title = ''
    return {'Título': title}

In [11]:
captura_titulo(soup)

{'Título': 'A EVOLUÇÃO DOS RELATÓRIOS DE SUSTENTABILIDADE DE EMPRESAS BRASILEIRAS DO SETOR DE ENERGIA ELÉTRICA'}

In [12]:
def captura_data(soup):
    try:
        date_tag = soup.find('div', {'class': 'item published'})
        date = date_tag.find('div', {'class': 'value'}).text
        date = ' '.join(date.split())
    except:
        date = ''
    return {'Data de Publicação': date[:10]}

In [13]:
captura_data(soup)

{'Data de Publicação': '2013-04-26'}

In [14]:
def captura_resumo(soup):
    try:
        abstract_section = soup.find('section', {'class': 'item abstract'})
        abstract = abstract_section.text
        abstract = abstract.replace('Resumo:', '')
        abstract = abstract.replace('Resumo', '').strip()
        #abstract = ' '.join(abstract.split())
    except:
        abstract = ''
    return {'Resumo': abstract}

In [15]:
captura_resumo(soup)

{'Resumo': 'Nos últimos anos, diante das pressões dos stakeholders, as empresas buscam maior transparência na divulgação de suas informações, dessa forma, além de aspectos econômicos, passaram a relatar ações associadas à sociedade e ao meio-ambiente por meio de Relatórios de Sustentabilidade. Devido a diversos fatores, a publicação desse documento tem sido uma prática cada vez mais frequente entre as empresas. Contudo, no que diz respeito a qualidade da informação, verifica-se que os relatórios apresentam diferentes níveis de profundidade. Dessa forma, o presente trabalho tem por objetivo verificar a evolução, nos relatórios de sustentabilidade, do grau de aderência plena e do grau de evidenciação efetiva aos indicadores de desempenho econômico, ambiental e social estabelecidos no modelo “G3” das Diretrizes para Elaboração de Relatórios de Sustentabilidade da Global Reporting Initiative (GRI), de empresas brasileiras do setor de energia elétrica no período de 2006 à 2009.  A partir da

In [16]:
def captura_palavras_chave(soup):
    try:
        kw_section = soup.find('section', {'class': 'item keywords'})
        kw = kw_section.text.replace('Palavras-chave:', '')
        kw = ' '.join(kw.split())
        kw = kw.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    if kw == '':
        try:
            kw_section = soup.find('section', {'class': 'item abstract'})
            p_list = kw_section.find_all('p')
            for p in p_list:
                if 'Palavras-chave' in p.text:
                    kw_full = p.text
                    kw = kw_full.replace('Palavras-chave', '').replace(':', '')
                    kw = ' '.join(kw.split())
                    kw = kw.replace(',', '.').replace(';', '.')
        except:
            pass
    return {'Palavras-chave': kw}

In [17]:
captura_palavras_chave(soup)

{'Palavras-chave': 'Relatório de sustentabilidade. GRI. Indicadores de desempenho. Setor de energia elétrica'}

In [18]:
def captura_autores(soup):
    autores = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        autores = authors_ul.find_all('span', {'class': 'name'})
        autores = [' '.join(a.text.split()) for a in autores]
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [19]:
captura_autores(soup)

{'Qtd. de Autores': 3,
 'Autor 1': 'Heloisa Maria Motta Morisue',
 'Autor 2': 'Maisa de Souza Ribeiro',
 'Autor 3': 'Isabela Alves de Morais Penteado',
 'Autor 4': '',
 'Autor 5': '',
 'Autor 6': ''}

In [20]:
def captura_afiliacao(soup):
    afiliacao = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        afiliacao = authors_ul.find_all('span', {'class': 'affiliation'})
        afiliacao = [' '.join(a.text.split()) for a in afiliacao]
        afiliacao = [limpar_afiliacao(a) for a in afiliacao]
    except:
        pass
    afiliacao = completar_lista(afiliacao)
    
    key_affiliation = [f"Afiliação {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, afiliacao))

In [21]:
captura_afiliacao(soup)

{'Afiliação 1': 'FEA-RP',
 'Afiliação 2': 'FEA-RP',
 'Afiliação 3': 'FEA-RP',
 'Afiliação 4': '',
 'Afiliação 5': '',
 'Afiliação 6': ''}

In [22]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Contabilidade Vista & Revista'}
    url_search_text = 'https://revistas.face.ufmg.br/index.php/contabilidadevistaerevista/search/search?query=__query__&searchJournal=1&authors=&title=&abstract=&galleyFullText=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&orderBy=score&orderDir=desc&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y-%m-%d')
    return df

## Pesquisa por artigos

In [23]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head(3)

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2013-04-26,A EVOLUÇÃO DOS RELATÓRIOS DE SUSTENTABILIDADE ...,"Nos últimos anos, diante das pressões dos stak...",Relatório de sustentabilidade. GRI. Indicadore...,3,Heloisa Maria Motta Morisue,Maisa de Souza Ribeiro,Isabela Alves de Morais Penteado,,,,FEA-RP,FEA-RP,FEA-RP,,,,https://revistas.face.ufmg.br/index.php/contab...,Contabilidade Vista & Revista
1,2021-03-02,GESTÃO DE CUSTOS AMBIENTAIS EM HOSPITAIS PRIVA...,O objetivo do estudo é identificar os procedim...,Contabilidade Ambiental. Gestão de Custos Ambi...,3,Marcio Atti Provenzano,Marcos Antônio de Souza,Débora Gomes de Gomes,,,,Unisinos,Universidade Federal de Uberlândia,Universidade Federal do Rio Grande – FURG,,,,https://revistas.face.ufmg.br/index.php/contab...,Contabilidade Vista & Revista
2,2017-05-04,GESTÃO SOCIOAMBIENTAL EM INSTITUIÇÕES BANCÁRIA...,"No ambiente empresarial, a reflexão sobre o te...",Gestão Ambiental. Indicadores. Sustentabilidade,4,Flávio José de Melo,Daniele Cristina Bernd,Marcos Wagner da Fonseca,Jorge Eduardo Scarpin,,,Universidade Federal do Paraná,Universidade Federal do Paraná,Universidade Federal do Paraná,Universidade Federal do Paraná,,,https://revistas.face.ufmg.br/index.php/contab...,Contabilidade Vista & Revista


In [24]:
df.to_csv('../data/RVC_UFMG.csv', index=False)

In [25]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2013-04-26,A EVOLUÇÃO DOS RELATÓRIOS DE SUSTENTABILIDADE ...,"Nos últimos anos, diante das pressões dos stak...",Relatório de sustentabilidade. GRI. Indicadore...,3,Heloisa Maria Motta Morisue,Maisa de Souza Ribeiro,Isabela Alves de Morais Penteado,,,,FEA-RP,FEA-RP,FEA-RP,,,,https://revistas.face.ufmg.br/index.php/contab...,Contabilidade Vista & Revista
1,2021-03-02,GESTÃO DE CUSTOS AMBIENTAIS EM HOSPITAIS PRIVA...,O objetivo do estudo é identificar os procedim...,Contabilidade Ambiental. Gestão de Custos Ambi...,3,Marcio Atti Provenzano,Marcos Antônio de Souza,Débora Gomes de Gomes,,,,Unisinos,Universidade Federal de Uberlândia,Universidade Federal do Rio Grande – FURG,,,,https://revistas.face.ufmg.br/index.php/contab...,Contabilidade Vista & Revista
2,2017-05-04,GESTÃO SOCIOAMBIENTAL EM INSTITUIÇÕES BANCÁRIA...,"No ambiente empresarial, a reflexão sobre o te...",Gestão Ambiental. Indicadores. Sustentabilidade,4,Flávio José de Melo,Daniele Cristina Bernd,Marcos Wagner da Fonseca,Jorge Eduardo Scarpin,,,Universidade Federal do Paraná,Universidade Federal do Paraná,Universidade Federal do Paraná,Universidade Federal do Paraná,,,https://revistas.face.ufmg.br/index.php/contab...,Contabilidade Vista & Revista
3,2013-04-26,Uma Análise comparativa da Responsabilidade So...,Este artigo objetiva estudar a Responsabilidad...,Responsabilidade Social Corporativa. Bancos. B...,3,Vicente Lima Crisóstomo,Fátima de Souza Freire,Patrícia Matias Soares,,,,Universidade Federal do Ceará,Universidade de Brasília,Universidade Federal do Ceará,,,,https://revistas.face.ufmg.br/index.php/contab...,Contabilidade Vista & Revista
4,2019-03-14,PERFIL DE ARRECADAÇÃO DOS MUNICÍPIOS DO ESTADO...,A Lei de Responsabilidade Fiscal surgiu com o ...,Receitas Públicas Próprias. Transferências Int...,3,Elizete Aparecida de Magalhães Wakim,Vasconcelos Reis Wakim,João Eustáquio de Lima,,,,Universidade Federal dos Vales Jequitinhonha e...,Universidade Federal dos Vales Jequitinhonha e...,Universidade Federal de Viçosa,,,,https://revistas.face.ufmg.br/index.php/contab...,Contabilidade Vista & Revista
5,2020-07-12,EVIDENCIAÇÃO DAS AÇÕES DIANTE DAS MUDANÇAS CLI...,O objetivo deste trabalho foi propor um modelo...,Evidenciação. Mudanças Climáticas. Determinant...,3,Juliano Almeida de Faria,José Célio Silveira Andrade,Sônia Maria da Silva Gomes,,,,Universidade Federal de Sergipe,Universidade Federal da Bahia,Universidade Federal da Bahia,,,,https://revistas.face.ufmg.br/index.php/contab...,Contabilidade Vista & Revista
6,2015-05-11,Fatores que Influenciam o Processo Ensino-Apre...,O estudo objetiva verificar a percepção dos di...,Educação e Pesquisa em Contabilidade,2,Franciele Beck,Rita Buzzi Rausch,,,,,Universidade Regional de Blumenau,Universidade Regional de Blumenau,,,,,https://revistas.face.ufmg.br/index.php/contab...,Contabilidade Vista & Revista
7,2020-07-12,"GÊNERO, ETNIA E RAÇA: DÉBITO OU CRÉDITO NA CON...",Esta pesquisa tem por objetivo verificar se ex...,Gênero. Etnia. Raça. Contabilidade. Discrimina...,3,José Luiz Borsatto Junior,Evellyn Danielly Zabotti,Maria da Piedade Araújo,,,,Universidade Paranaense,Universidade Estadual do Oeste do Paraná,,,,,https://revistas.face.ufmg.br/index.php/contab...,Contabilidade Vista & Revista
8,2020-12-05,O PERCURSO DO CONTRATO DE LEGITIMAÇÃO PARA O D...,Os bancos de desenvolvimento atuam concedendo ...,desenvolvimento sustentável. relato integrado....,3,Kelli Juliane Favato,Marguit Neumann,Simone Leticia Raimundini Sanches,,,,Universidade Estadual de Maringá,Universidade Estadual de Maringá,Universidade Estadual de Maringá,,,,https://revistas.face.ufmg.br/index.php/contab...,Contabilidade Vista & Revista
